<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Лемматизация-и-предобработка-текста" data-toc-modified-id="Лемматизация-и-предобработка-текста-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Лемматизация и предобработка текста</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DesicionTreeClassifier" data-toc-modified-id="DesicionTreeClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>DesicionTreeClassifier</a></span></li><li><span><a href="#LinearSVC" data-toc-modified-id="LinearSVC-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LinearSVC</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Тестовая-выборка" data-toc-modified-id="Тестовая-выборка-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Тестовая выборка</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000)
import numpy as np
import re
import time
import catboost as cb

# Лемматизация и предобработка
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV

#Модели

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df.head(10)

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0
5,"""\n\nCongratulations from me as well, use the tools well. · talk """,0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0
8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",0
9,alignment on this subject and which are contrary to those of DuLithgow,0


Требуется лемматизация и очистка текста от лишних символов для перевода их к векторному виду

In [5]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Классы несбалансированы соотношение 1 к 10,поэтому при обучении моделей надо обязательно использовать сбалансированное взвешивание классов

### Лемматизация и предобработка текста

Напишем функцию для этого

In [3]:
m = Mystem()

In [4]:
def lemmatize_text(text):    
    text = text.lower()
    lemm_text = "".join(m.lemmatize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

Теперь лемматизируем наш текст

In [5]:
%%time
df['text'] = df['text'].apply(lemmatize_text)

CPU times: user 36 s, sys: 7.57 s, total: 43.5 s
Wall time: 1min 37s


In [9]:
df['text'].head()

0                                                                                                                                                                                                                                                                                                                                                                    explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalisms just closure on some gas after i voted at new york dolls fac and please don t remove the template from the talk page since i m retired now
1                                                                                                                                                                                                                                                                                                                                                                                                         

Поскольку нельзя переводить в векторы всю выборку(из-за утечки данных),сделаем это до векторизации признаков

In [6]:
target = df['toxic']
features = df.drop(['toxic'], axis=1)

In [7]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                              target, 
                                                                              test_size=.3, 
                                                                              random_state=1234)

Посмотрим корректность разбиения

In [8]:
print("Признаки")
print(f'Обучающая: {features_train.shape} Тестовая: {features_test.shape}')
print('Таргет')
print(f'Обучающая: {target_test.shape}  Тестовая: {target_test.shape}')

Признаки
Обучающая: (111699, 1) Тестовая: (47872, 1)
Таргет
Обучающая: (47872,)  Тестовая: (47872,)


Выборки разделены нормально, можно приступать к векторизации

Добавим английский корпус со стоп-словами из библиотеки nltk

In [9]:
stopwords = set(stopwords.words('english'))

In [10]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

Векторизируем признаки для обучающей выборки,обучая векторизатор только на тренировачной выборке

In [11]:
features_train = count_tf_idf.fit_transform(features_train['text'])
features_test = count_tf_idf.transform(features_test['text'])

Признаки подготовлены, можно приступать к обучению моделей

## Обучение

### LogisticRegression

In [16]:
%%time
model_lr = LogisticRegression(random_state = 1234, class_weight='balanced')
grid_params= {'penalty': ['l1','l2'], 
              'C' : [0.5, 1.0, 5.0, 15.0],
              'solver': ['newton-cg', 'lbfgs', 'liblinear'],
              'intercept_scaling' : range(5, 100, 40)}
lr_gs = GridSearchCV(model_lr,cv=3,param_grid = grid_params,n_jobs=-1,scoring='f1')
lr_gs.fit(features_train,target_train)
print(f'Лучшие параметры: {lr_gs.best_params_}')
print(f'Лучший результат:{lr_gs.best_score_}')


Лучшие параметры: {'C': 5.0, 'intercept_scaling': 5, 'penalty': 'l2', 'solver': 'lbfgs'}
Лучший результат:0.7694298872716896
CPU times: user 45min 35s, sys: 25min 15s, total: 1h 10min 51s
Wall time: 1h 10min 58s


Лучшие параметры: {'C': 5.0, 'intercept_scaling': 5, 'penalty': 'l2', 'solver': 'lbfgs'}
Лучший результат:0.7694298872716896
CPU times: user 45min 35s, sys: 25min 15s, total: 1h 10min 51s
Wall time: 1h 10min 58s

### DesicionTreeClassifier

In [13]:
%%time
dtc = DecisionTreeClassifier(random_state=1234,class_weight = 'balanced')
grid_params ={'max_depth': range(10, 131, 20),
              'max_leaf_nodes': range(10, 61, 20)}
dtc_gs = GridSearchCV(dtc,cv=3,param_grid = grid_params,n_jobs=-1,scoring='f1')
dtc_gs.fit(features_train,target_train)
print(f'Лучшие параметры: {dtc_gs.best_params_}')
print(f'Лучший результат:{dtc_gs.best_score_}')

Лучшие параметры: {'max_depth': 30, 'max_leaf_nodes': 50}
Лучший результат:0.6294799682091264
CPU times: user 1h 8min 35s, sys: 3.32 s, total: 1h 8min 38s
Wall time: 1h 8min 41s


Лучшие параметры: {'max_depth': 30, 'max_leaf_nodes': 50}
Лучший результат:0.6294799682091264
CPU times: user 1h 8min 35s, sys: 3.32 s, total: 1h 8min 38s
Wall time: 1h 8min 41s

### LinearSVC

In [18]:
%%time
svc = LinearSVC(random_state = 1234,class_weight ='balanced')
grid_params={'penalty': ['l1','l2'],
             'C': np.logspace(-5,5,7),
             'max_iter': [100,300,500]}
svc_gs = GridSearchCV(svc,cv=3,param_grid = grid_params,n_jobs=-1,scoring='f1')
svc_gs.fit(features_train,target_train)
print(f'Лучшие параметры: {svc_gs.best_params_}')
print(f'Лучший результат:{svc_gs.best_score_}')

Лучшие параметры: {'C': 1.0, 'max_iter': 300, 'penalty': 'l2'}
Лучший результат:0.7633296827075225
CPU times: user 1min 35s, sys: 1.21 s, total: 1min 37s
Wall time: 1min 37s


Лучшие параметры: {'C': 1.0, 'max_iter': 300, 'penalty': 'l2'}
Лучший результат:0.7633296827075225
CPU times: user 1min 35s, sys: 1.21 s, total: 1min 37s
Wall time: 1min 37s


### RandomForestClassifier

In [14]:
%%time
rfc = RandomForestClassifier(random_state = 1234,class_weight ='balanced')
grid_params ={  'n_estimators': range(10, 121, 20),
                'max_depth': range(10, 131, 20),
                'criterion' : ['gini', 'entropy', 'log_loss']}
rfc_gs = GridSearchCV(rfc,cv=3,param_grid = grid_params,n_jobs=-1,scoring='f1')
rfc_gs.fit(features_train,target_train)
print(f'Лучшие параметры: {rfc_gs.best_params_}')
print(f'Лучший результат:{rfc_gs.best_score_}')

Лучшие параметры: {'criterion': 'entropy', 'max_depth': 130, 'n_estimators': 110}
Лучший результат:0.5672398003364928
CPU times: user 4h 11min 31s, sys: 18 s, total: 4h 11min 49s
Wall time: 4h 11min 51s


Лучшие параметры: {'criterion': 'entropy', 'max_depth': 130, 'n_estimators': 110}
Лучший результат:0.5672398003364928
CPU times: user 4h 11min 31s, sys: 18 s, total: 4h 11min 49s
Wall time: 4h 11min 51s

### Выводы

**Logistic Regression**

- Лучшие параметры: {'C': 5.0, 'intercept_scaling': 5, 'penalty': 'l2', 'solver': 'lbfgs'}
- Лучший результат F1:**0.769**
- CPU times: user 45min 35s, sys: 25min 15s, total: 1h 10min 51s
- Wall time: 1h 10min 58s

**Desicion Tree Classifier**

- Лучшие параметры: {'max_depth': 30, 'max_leaf_nodes': 50}
- Лучший результат:**0.629**
- CPU times: user 1h 8min 35s, sys: 3.32 s, total: 1h 8min 38s
- Wall time: 1h 8min 41s

*Данную модель мы не будем использовать для проверки на тестовой выборке ,т.к. она не проходит по минимальным требованиям метрики*

**LinearSVC**

- Лучшие параметры: {'C': 1.0, 'max_iter': 300, 'penalty': 'l2'}
- Лучший результат F1:**0.763**
- CPU times: user 1min 35s, sys: 1.21 s, total: 1min 37s
- Wall time: 1min 37s

**Random Forest Classifier**

- Лучшие параметры: {'criterion': 'entropy', 'max_depth': 130, 'n_estimators': 110}
- Лучший результат:**0.567**
- CPU times: user 4h 11min 31s, sys: 18 s, total: 4h 11min 49s
- Wall time: 4h 11min 51s

*Данную модель мы не будем использовать для проверки на тестовой выборке ,т.к. она не проходит по минимальным требованиям метрики*


### Тестовая выборка

In [30]:
model_test_lr= LogisticRegression(random_state = 1234, class_weight='balanced',C = 5.0, 
                                  intercept_scaling = 5, penalty = 'l2', solver = 'lbfgs')
model_test_lr.fit(features_train,target_train)

LogisticRegression(C=5.0, class_weight='balanced', intercept_scaling=5,
                   random_state=1234)

In [39]:
predictions_linear = model_test_lr.predict(features_test)
f1 = f1_score(target_test,predictions_linear)
print(f'F1 LogisticRegression: {f1}')

F1 LogisticRegression: 0.7674508291554185


In [40]:
svc_test =  LinearSVC(random_state = 1234,class_weight ='balanced',
                      C= 1.0, max_iter = 300, penalty ='l2')
svc_test.fit(features_train,target_train)
predictions_svc = svc_test.predict(features_test)
f1 = f1_score(target_test,predictions_svc)
print(f'F1 LogisticRegression: {f1}')

F1 LogisticRegression: 0.7615959119496856


<div class="alert alert-block alert-success">
<b>Успех:</b> Тестирование было сделано верно!
</div>

## Выводы

- Для обучения и использования подходят обе линейные модели, LogisticRegressor и LinearSVC. 
- Последняя обучается значительно быстрее, при этом результат у нее получился практически такой же
- Для больших выборок я бы использовал LinearCSV
- Сложные ансамблевые модели обучаются долго даже на такой выборке и дают более низкий результат для данного набора данных
- Дерево решений так же не показало достаточного значения F1 для дальнейшей реализации

**Значения F1**

 - **Logistic Regression** - 0.767
 - **Linear SVC**  - 0.761
 
 Обе модели удовлетворяют условию задания